In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


Using cuda device


In [5]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = nn.functional.one_hot(y, num_classes=10).float()
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

#        if batch % 100 == 0:
#            loss, current = loss.item(), (batch + 1) * len(X)
#            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def eval(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            y = nn.functional.one_hot(y, num_classes=10).float()
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"\t Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    
    return correct

# Learning the model parameters

In [ ]:
import optuna

epochs = 100
def objective(trial):
    
    learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.1, log=True)
#    weight_decay = trial.suggest_float("weight_decay", 1.0e-10, 1.0e-2, log=True)
    
    model = NeuralNetwork().to(device)
    # print(model)

    loss = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for t in range(epochs):
        print(f"Epoch {t+1}")
        train(train_dataloader, model, loss, optimizer)
#        print("Training Data")
#        tain_err = test(train_dataloader, model1, loss_ce)
#        print("Test Data")
        test_err = eval(test_dataloader, model, loss)
        print('-----------------------------')
    print("Done!")
    
    return test_err

In [8]:
ntrial = 50
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials = ntrial)

[I 2025-01-26 15:52:39,345] A new study created in memory with name: no-name-bbd52e55-9f11-4ca5-a924-4ef1a95df261


Epoch 1
	 Accuracy: 20.2%, Avg loss: 0.093766
-----------------------------
Epoch 2
	 Accuracy: 25.3%, Avg loss: 0.088740
-----------------------------
Epoch 3
	 Accuracy: 31.9%, Avg loss: 0.085878
-----------------------------
Epoch 4
	 Accuracy: 37.8%, Avg loss: 0.083904
-----------------------------
Epoch 5
	 Accuracy: 46.0%, Avg loss: 0.082302
-----------------------------
Epoch 6
	 Accuracy: 52.4%, Avg loss: 0.080866
-----------------------------
Epoch 7
	 Accuracy: 56.2%, Avg loss: 0.079522
-----------------------------
Epoch 8
	 Accuracy: 58.4%, Avg loss: 0.078242
-----------------------------
Epoch 9
	 Accuracy: 60.1%, Avg loss: 0.077018
-----------------------------
Epoch 10
	 Accuracy: 61.2%, Avg loss: 0.075844
-----------------------------
Epoch 11
	 Accuracy: 62.1%, Avg loss: 0.074719
-----------------------------
Epoch 12
	 Accuracy: 62.6%, Avg loss: 0.073643
-----------------------------
Epoch 13
	 Accuracy: 63.1%, Avg loss: 0.072617
-----------------------------
Epoch 14

[I 2025-01-26 16:09:22,405] Trial 0 finished with value: 0.7258 and parameters: {'learning_rate': 0.00044061159017729987}. Best is trial 0 with value: 0.7258.


	 Accuracy: 72.6%, Avg loss: 0.044683
-----------------------------
Done!
Epoch 1
	 Accuracy: 64.5%, Avg loss: 0.067868
-----------------------------
Epoch 2
	 Accuracy: 67.2%, Avg loss: 0.057313
-----------------------------
Epoch 3
	 Accuracy: 69.1%, Avg loss: 0.051420
-----------------------------
Epoch 4
	 Accuracy: 70.9%, Avg loss: 0.047782
-----------------------------
Epoch 5
	 Accuracy: 72.3%, Avg loss: 0.045355
-----------------------------
Epoch 6
	 Accuracy: 73.6%, Avg loss: 0.043585
-----------------------------
Epoch 7
	 Accuracy: 74.7%, Avg loss: 0.042190
-----------------------------
Epoch 8
	 Accuracy: 75.7%, Avg loss: 0.041025
-----------------------------
Epoch 9
	 Accuracy: 76.5%, Avg loss: 0.040018
-----------------------------
Epoch 10
	 Accuracy: 77.1%, Avg loss: 0.039129
-----------------------------
Epoch 11
	 Accuracy: 77.8%, Avg loss: 0.038336
-----------------------------
Epoch 12
	 Accuracy: 78.3%, Avg loss: 0.037622
-----------------------------
Epoch 13
	 

[I 2025-01-26 16:25:27,147] Trial 1 finished with value: 0.8533 and parameters: {'learning_rate': 0.008307234554315588}. Best is trial 1 with value: 0.8533.


...

	 Accuracy: 88.6%, Avg loss: 0.018187
-----------------------------
Done!
Epoch 1
	 Accuracy: 19.9%, Avg loss: 0.093554
-----------------------------
Epoch 2
	 Accuracy: 23.8%, Avg loss: 0.090435
-----------------------------
Epoch 3
	 Accuracy: 27.5%, Avg loss: 0.088270
-----------------------------
Epoch 4
	 Accuracy: 30.9%, Avg loss: 0.086655
-----------------------------
Epoch 5
	 Accuracy: 34.9%, Avg loss: 0.085356
-----------------------------
Epoch 6
	 Accuracy: 39.5%, Avg loss: 0.084242
-----------------------------
Epoch 7
	 Accuracy: 43.9%, Avg loss: 0.083239
-----------------------------
Epoch 8
	 Accuracy: 47.6%, Avg loss: 0.082309
-----------------------------
Epoch 9
	 Accuracy: 49.9%, Avg loss: 0.081428
-----------------------------
Epoch 10
	 Accuracy: 51.9%, Avg loss: 0.080587
-----------------------------
Epoch 11
	 Accuracy: 53.4%, Avg loss: 0.079778
-----------------------------
Epoch 12
	 Accuracy: 54.9%, Avg loss: 0.078997
-----------------------------
Epoch 13
	 

[I 2025-01-27 05:03:37,868] Trial 48 finished with value: 0.696 and parameters: {'learning_rate': 0.0002633663387350195}. Best is trial 15 with value: 0.8943.


	 Accuracy: 69.6%, Avg loss: 0.050245
-----------------------------
Done!
Epoch 1
	 Accuracy: 70.2%, Avg loss: 0.049195
-----------------------------
Epoch 2
	 Accuracy: 74.6%, Avg loss: 0.041882
-----------------------------
Epoch 3
	 Accuracy: 77.0%, Avg loss: 0.038487
-----------------------------
Epoch 4
	 Accuracy: 79.0%, Avg loss: 0.036216
-----------------------------
Epoch 5
	 Accuracy: 80.1%, Avg loss: 0.034561
-----------------------------
Epoch 6
	 Accuracy: 80.8%, Avg loss: 0.033281
-----------------------------
Epoch 7
	 Accuracy: 81.3%, Avg loss: 0.032244
-----------------------------
Epoch 8
	 Accuracy: 81.7%, Avg loss: 0.031385
-----------------------------
Epoch 9
	 Accuracy: 82.2%, Avg loss: 0.030655
-----------------------------
Epoch 10
	 Accuracy: 82.5%, Avg loss: 0.030017
-----------------------------
Epoch 11
	 Accuracy: 82.8%, Avg loss: 0.029446
-----------------------------
Epoch 12
	 Accuracy: 83.1%, Avg loss: 0.028937
-----------------------------
Epoch 13
	 

[I 2025-01-27 05:19:44,180] Trial 49 finished with value: 0.8817 and parameters: {'learning_rate': 0.029551896288683436}. Best is trial 15 with value: 0.8943.


	 Accuracy: 88.2%, Avg loss: 0.019252
-----------------------------
Done!


In [9]:
print(study.best_params)
print(study.best_value)

{'learning_rate': 0.09524803610407732}
0.8943


## Re-training the best model

In [ ]:
learning_rate = study.best_params['learning_rate']
print("learning_rate = ", learning_rate)

model = NeuralNetwork().to(device)
# print(model)

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}")
    train(train_dataloader, model, loss, optimizer)
    print("Training Data")
    tain_err = eval(train_dataloader, model, loss)
    print("Test Data")
    test_err = eval(test_dataloader, model, loss)
    print('-----------------------------')
print("Done!")

learning_rate =  0.09524803610407732
Epoch 1
Training Data
	 Accuracy: 78.2%, Avg loss: 0.037841
Test Data
	 Accuracy: 77.0%, Avg loss: 0.038509
-----------------------------
Epoch 2
Training Data
	 Accuracy: 81.8%, Avg loss: 0.032547
Test Data
	 Accuracy: 80.6%, Avg loss: 0.033470
-----------------------------
Epoch 3
Training Data
	 Accuracy: 83.0%, Avg loss: 0.030016
Test Data
	 Accuracy: 82.0%, Avg loss: 0.031127
-----------------------------
Epoch 4
Training Data
	 Accuracy: 83.8%, Avg loss: 0.028268
Test Data
	 Accuracy: 82.7%, Avg loss: 0.029504
-----------------------------
Epoch 5
Training Data
	 Accuracy: 84.6%, Avg loss: 0.026896
Test Data
	 Accuracy: 83.3%, Avg loss: 0.028250
-----------------------------
Epoch 6
Training Data
	 Accuracy: 85.2%, Avg loss: 0.025753
Test Data
	 Accuracy: 83.8%, Avg loss: 0.027212
-----------------------------
Epoch 7
Training Data
	 Accuracy: 85.6%, Avg loss: 0.024858
Test Data
	 Accuracy: 84.3%, Avg loss: 0.026420
---------------------------

In [11]:
torch.save(model.state_dict(), "./trained_models/MLP_FashionMNIST_MSE.pth")
print("Saved PyTorch Model State to ./trained_models/MLP_FashionMNIST_MSE.pth")

Saved PyTorch Model State to ./trained_models/MLP_FashionMNIST_MSE.pth
